# Purpose

This notebook demonstrates the data pipeline from raw tables to analytical datasets. At the end of this activity, train & test data sets are created from raw data.

## Imports

In [0]:
from ta_lib.core.api import get_package_path
PACKAGE_PATH=get_package_path()

In [0]:
import sys
import os.path as op
CURR_LOC_IN_DBFS=op.join(PACKAGE_PATH,"..","databricks/03_reference_notebooks/python/")
sys.path.append(CURR_LOC_IN_DBFS)

In [0]:
from pprint import pprint
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.options.mode.use_inf_as_na = True


In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
# standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer,
    display_as_tabs, string_cleaning, merge_info, initialize_environment,
    list_datasets, load_dataset, save_dataset
)
import ta_lib.eda.api as eda

In [0]:
initialize_environment(debug=False, hide_warnings=True)

## Utility functions

# 1. Initialization

In [0]:
config_path = op.join(CURR_LOC_IN_DBFS,"conf","config.yml")
context = create_context(config_path)
pprint(list_datasets(context))

orders_df = load_dataset(context, 'raw/orders')
prod_df = load_dataset(context, 'raw/product')

# 2. Data cleaning and consolidation

**<u>NOTES</u>**

The focus here is to create a cleaned dataset that is appropriate for solving the DS problem at hand from the raw data.

**1. Do**
* clean dataframe column names
* ensure dtypes are set properly
* join with other tables etc to create features
* transform, if appropriate, datetime like columns to generate additional features (weekday etc)
* transform, if appropriate, string columns to generate additional features
* discard cols that are not useful for training the model (IDs, constant cols, duplicate cols etc)
* additional features generated from existing columns


**2. Don't**
* handle missing values or outliers here. mark them and leave them for processing downstream.

## 2.1 Clean individual tables

### Products Table

From data discovery, we know the following

* all columns are strings : nothing to fix. Apply generic cleaning (strip extra whitespace etc)
* ensure all `invalid` string entries are mapped to np.NaN
* some column are duplicates (eg. color, Ext_Color). Better to `coalesce` them instead of an outright discard of one of the columns.
* SKU is key column : ensure no duplicate values
* This will go into production code

In [0]:
prod_df_clean = (
    prod_df
    # while iterating on testing, it's good to copy the dataset(or a subset)
    # as the following steps will mutate the input dataframe. The copy should be
    # removed in the production code to avoid introducing perf. bottlenecks.
    .copy()

    # set dtypes : nothing to do here
    .passthrough()

    .transform_columns(prod_df.columns.to_list(), string_cleaning, elementwise=False)
    
    .replace({'': np.NaN})
    
    # drop unnecessary cols : nothing to do here
    .coalesce(['color', 'Ext_Color'], 'color', delete_columns=True)
    
    # drop unnecessary cols : nothing to do here
    .coalesce(['MemorySize', 'Ext_memorySize'], 'memory_size', delete_columns=True)
    
    # ensure that the key column does not have duplicate records
    .remove_duplicate_rows(col_names=['SKU'], keep_first=True)
    
    # clean column names (comment out this line while cleaning data above)
    .clean_names(case_type='snake')
)
prod_df_clean.head()

### NOTE

It's always a good idea to save cleaned tabular data using a storage format that supports the following 

1. preserves the type information
2. language agnostic storage format
3. Supports compression
4. Supports customizing storage to optimize different data access patterns

For larger datasets, the last two points become crucial.

`Parquet` is one such file format that is very popular for storing tabular data. It has some nice properties:
- Similar to pickles & RDS datasets, but compatible with all languages
- Preserves the datatypes
- Compresses the data and reduces the filesize
- Good library support in Python and other languages
- As a columnar storage we can efficiently read fewer columns
- It also supports chunking data by groups of columns (for instance, by dates or a particular value of a key column) that makes loading subsets of the data fast.

In [0]:
save_dataset(context, prod_df_clean, 'cleaned/product')

### Orders Table

From data discovery, we know the following

* key columns: None
* integer columns: Quantity, InvoiceNo, Orderno, Quantity
* datetime columns: LedgerDate
* This will go into production code

In [0]:
# column names after cleaning

str_cols = list(
    set(orders_df.select_dtypes('object').columns.to_list()) 
    - set(['Customername', 'InvoiceNo','Quantity', 'InvoiceNo', 'Orderno', 'LedgerDate'])
)
orders_df_clean = (
    orders_df
    
    .copy()
    #.sample(frac=1, resample=False)

    # set dtypes
    .change_type(['Quantity', 'InvoiceNo', 'Orderno'], np.int64)
    
    # set dtypes
    .to_datetime('LedgerDate', format='%d/%m/%Y')
    
    # clean string columns (NOTE: only after setting datetime)
    .transform_columns(str_cols, string_cleaning, elementwise=False)

    # clean column names                                                                                                                                   
    .clean_names(case_type='snake')
    .rename_columns({'orderno': 'order_no'})
)
orders_df_clean.head().T


In [0]:
save_dataset(context, orders_df_clean, 'cleaned/orders')

## 2.2 Create consolidated features table

Here we take the cleaned dataframes and merge them to form the consolidated table.

We know from data discovery that `products` is a dimension table and `orders` is a fact table, so we want to do a left join here. * This will go into production code

In [0]:
sales_df = pd.merge(orders_df_clean, prod_df_clean, how='inner', on='sku', validate='m:1')
merge_info(orders_df_clean, prod_df_clean, sales_df)

## 2.3 Business intuition features

This section can go into production code if these features are used in final model

#### First Time Customer.
- A binary feature that tells if the customer is in business for the first time or not.

In [0]:
# first time customer
cust_details = sales_df.groupby(['customername']).agg({'ledger_date':'min'}).reset_index()
cust_details.columns = ['customername','ledger_date']
cust_details['first_time_customer'] = 1
sales_df = sales_df.merge(cust_details, on=['customername','ledger_date'], how='left')
sales_df['first_time_customer'].fillna(0, inplace=True)

#### Days Since Last Purchase of a customer
- Feature representing the number of days from the last purchase of a customer. 
- Quantifies the Gaps customers take b/w purchases

In [0]:
#### days since last purchase
sales_df.sort_values('ledger_date',inplace=True)
sales_df['days_since_last_purchase'] = (
    sales_df
       .groupby('customername')['ledger_date']
       .diff()
       .dt.days
       .fillna(0, downcast='infer'))

In [0]:
# create a sample dataframe with minimal processing

sales_df_processed = (
    sales_df
    
    # tweak to test pipeline quickly or profile performance
    #.sample(frac=1, replace=False)
    
    # any additional processing/cleaning
)

# Any verifications on the data
from ta_lib.eda.api import get_variable_summary
displayHTML(display_as_tabs([
    ("Summary", f"Length: {len(sales_df_processed)}, Columns: {len(sales_df_processed.columns)}"),
    ("Variable summary", get_variable_summary(sales_df_processed)),
    ("head", sales_df.head(5).T),
    ("tail", sales_df.tail(5).T),
], cloud_env='Databricks'))

In [0]:
save_dataset(context, sales_df_processed, 'cleaned/sales')

# 3. Generate Train, Validation and Test datasets

- We split the data into train, test (optionally, also a validation dataset)
- In this example, we are binning the target into 10 quantiles and then use a Stratified Shuffle to split the data.
- See sklearn documentation on the various available splitters
- https://scikit-learn.org/stable/modules/classes.html#splitter-classes
- This will go into production code (training only)

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit
from ta_lib.core.api import custom_train_test_split  # helper function to customize splitting
from scripts import *

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=context.random_seed)
sales_df_train, sales_df_test = custom_train_test_split(sales_df_processed, splitter, by=binned_selling_price)

In [0]:
target_col = "unit_price"

train_X, train_y = (
    sales_df_train
    
    # split the dataset to train and test
    .get_features_targets(target_column_names=target_col)
)
save_dataset(context, train_X, 'train/sales/features')
save_dataset(context, train_y, 'train/sales/target')


test_X, test_y = (
    sales_df_test
    
    # split the dataset to train and test
    .get_features_targets(target_column_names=target_col)
)
save_dataset(context, test_X, 'test/sales/features')
save_dataset(context, test_y, 'test/sales/target')